In [15]:
import math
import numpy as np
import re
import cv2
import glob

from collections import Counter
from matplotlib import pyplot as plt
from PIL import Image

PATH = "/home/nicholas/SHOTTA-FLOW-4-REMIX-FT.-BLUEFACE/src/training/pictures"
bigimageset = []

for filename in glob.glob(PATH + '/*.png'):
    im=cv2.imread(filename)
    bigimageset.append(np.array([im, filename[-8:-4]]))
    #print(filename[-8:-4])

#print(bigimageset[0][1])
def hotlocalsingles(char):
    index = ord(char)
    index = index - 48 if index < 58 else index - 55
    onehot = [0]*36
    onehot[index] = 1
    return onehot

lower_blue = np.array([100,100,30])
upper_blue = np.array([140,255,255])

chars = []
output = []

for miniarray in bigimageset:
    platehsv = cv2.cvtColor(miniarray[0], cv2.COLOR_BGR2HSV)
    platehsv = cv2.inRange(platehsv, lower_blue, upper_blue)
    
    _, morecnts, _ = cv2.findContours(platehsv, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boundRect = [None]*len(morecnts)
    contours_poly = [None]*len(morecnts)
    
    for i, c in enumerate(morecnts):
        contours_poly[i] = cv2.approxPolyDP(c, 3, True)
        boundRect[i] = cv2.boundingRect(contours_poly[i])
    
    np.sort(boundRect, axis=0)

    #np.delete(boundRect, 2)
    
    for i, c in enumerate(boundRect):
        if 300 > c[0] and 300 < c[0] + c[2]:
            pass
        elif c[2] > 120:
            chars.append(cv2.resize(platehsv[c[0]:c[0]+int(c[2]/2.0), c[1]:c[1]+c[3]], (106, 160)))
            chars.append(cv2.resize(platehsv[c[0]+int(c[2]/2.0):c[0]+c[2], c[1]:c[1]+c[3]], (106, 160)))
        else:
            chars.append(cv2.resize(platehsv[c[0]:c[0]+c[2], c[1]:c[1]+c[3]], (106, 160)))
    
    output.append(miniarray[1])

chars = np.array(chars)
output = np.array(output)
for i in chars:
    cv2.imshow("uh", i)

#print(str(chars.shape))
#print(str(output.shape))

error: /build/opencv-L2vuMj/opencv-3.2.0+dfsg/modules/imgproc/src/imgwarp.cpp:3492: error: (-215) ssize.width > 0 && ssize.height > 0 in function resize


In [14]:
from keras import layers
from keras import models
from keras import optimizers

from keras.utils import plot_model
from keras import backend

def reset_weights(model):
    session = backend.get_session()
    for layer in model.layers: 
        if hasattr(layer, 'kernel_initializer'):
            layer.kernel.initializer.run(session=session)
            
conv_model = models.Sequential()
conv_model.add(layers.Conv2D(32, (3, 3), activation='relu',
                             input_shape=(160, 106, 3)))
conv_model.add(layers.MaxPooling2D((2, 2)))
conv_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
conv_model.add(layers.MaxPooling2D((2, 2)))
conv_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
conv_model.add(layers.MaxPooling2D((2, 2)))
conv_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
conv_model.add(layers.MaxPooling2D((2, 2)))
conv_model.add(layers.Flatten())
conv_model.add(layers.Dropout(0.5))
conv_model.add(layers.Dense(512, activation='relu'))
conv_model.add(layers.Dense(36, activation='softmax'))

LEARNING_RATE = 1e-4
conv_model.compile(loss='categorical_crossentropy',
                   optimizer=optimizers.RMSprop(lr=LEARNING_RATE),
                   metrics=['acc'])

reset_weights(conv_model)

VALIDATION_SPLIT = 0.2

history_conv = conv_model.fit(chars, output, 
                              validation_split=VALIDATION_SPLIT, 
                              epochs=20, batch_size=16)

import sklearn
from sklearn import metrics

y_pred = conv_model.predict(chars)
y_pred = [np.argmax(i) for i in y_pred]
y_data = [np.argmax(i) for i in output]

confusion_matrix = sklearn.metrics.confusion_matrix(y_data, y_pred)
confusion_matrix = confusion_matrix.astype("float") / confusion_matrix.sum(axis=1)[:, np.newaxis]
print(confusion_matrix)

conv_model.save('plate.h5')

ValueError: Error when checking input: expected conv2d_17_input to have 4 dimensions, but got array with shape (12970, 160, 106)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history_conv.history['loss'])
plt.plot(history_conv.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train loss', 'val loss'], loc='upper right')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history_conv.history['acc'])
plt.plot(history_conv.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy (%)')
plt.xlabel('epoch')
plt.legend(['train accuracy', 'val accuracy'], loc='lower right')
plt.show()